In [1]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install pandas scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer

from sklearn.ensemble import RandomForestClassifier

import xgboost as xgb

In [4]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-06-trees/CreditScoring.csv'
df = pd.read_csv(data)
df.head()

,Status,Seniority,Home,Time,Age,Marital,Records,Job,Expenses,Income,Assets,Debt,Amount,Price
0,1,9,1,60,30,2,1,3,73,129,0,0,800,846
1,1,17,1,60,58,3,1,1,48,131,0,0,1000,1658
2,2,10,2,36,46,2,2,3,90,200,3000,0,2000,2985
3,1,0,1,60,24,1,1,1,63,182,2500,0,900,1325
4,1,0,1,36,26,1,1,1,46,107,0,0,310,910


In [5]:
df.columns = df.columns.str.lower()

status_values = { 1: 'ok', 2: 'default', 0: 'unk'}

df.status = df.status.map(status_values)

home_values = {1: 'rent',2: 'owner',3: 'private',4: 'ignore',5: 'parents',6: 'other',0: 'unk'}

df.home = df.home.map(home_values)

marital_values = { 1: 'single',2: 'married',3: 'widow',4: 'separated',5: 'divorced',0: 'unk'}

df.marital = df.marital.map(marital_values)

records_values = {1: 'no',2: 'yes',0: 'unk'}

df.records = df.records.map(records_values)

job_values = {1: 'fixed',2: 'partime',3: 'freelance', 4: 'others',0: 'unk'}

df.job = df.job.map(job_values)

for c in ['income', 'assets', 'debt']:
    df[c] = df[c].replace(to_replace=99999999, value=np.nan)

df = df[df.status != 'unk'].reset_index(drop=True)

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=11)

df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

y_train = (df_train.status == 'default').astype('int').values
y_test = (df_test.status == 'default').astype('int').values

del df_train['status']
del df_test['status']

In [7]:
dv = DictVectorizer(sparse=False)

train_dicts = df_train.fillna(0).to_dict(orient='records')
x_train = dv.fit_transform(train_dicts)

test_dicts = df_test.fillna(0).to_dict(orient='records')
x_test = dv.transform(test_dicts)

In [8]:
rf = RandomForestClassifier(n_estimators=200,
                            max_depth=10,
                            min_samples_leaf=3,
                            random_state=1)
rf.fit(x_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=3, n_estimators=200,
                       random_state=1)

In [9]:
!pip install ruamel-yaml

Defaulting to user installation because normal site-packages is not writeable


In [10]:
!pip install pywinpty==1.1.6

Defaulting to user installation because normal site-packages is not writeable


In [11]:
!pip install bentoml

Defaulting to user installation because normal site-packages is not writeable


In [15]:
print(bentoml.__version__)

1.0.7


In [13]:
import bentoml

bentoml.sklearn.save_model("random_forest_model", rf)

Model(tag="random_forest_model:57l7qqks42t2nlcq", path="C:\Users\mashk\bentoml\models\random_forest_model\57l7qqks42t2nlcq\")

In [20]:
#!dir C:\Users\mashk\bentoml\models\random_forest_model\57l7qqks42t2nlcq\

In [18]:
!pip install pydantic

Defaulting to user installation because normal site-packages is not writeable


In [21]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    name: str
    age: int
    country: str
    rating: float

In [25]:
#!curl -O https://s3.us-west-2.amazonaws.com/bentoml.com/mlzoomcamp/coolmodel.bentomodel

In [29]:
#!bentoml models import coolmodel.bentomodel

In [26]:
#!bentoml models list

In [27]:
#!bentoml models get mlzoomcamp_homework:qtzdz3sl

In [28]:
# bentofile.yaml
# bentoml build
# bentoml containerize mlzoomcamp_model:latest
# docker run -it --rm -p 3000:3000 mlzoomcamp_model:2jmj5jsqhoripugg serve --production".

In [30]:
!pip3 install locust

Defaulting to user installation because normal site-packages is not writeable
  Created wheel for Flask-BasicAuth: filename=Flask_BasicAuth-0.2.0-py3-none-any.whl size=4243 sha256=75566898903f083dc5f687759ed99de915e469512acf011f4a81bf1b0e2ca25c
  Stored in directory: c:\users\mashk\appdata\local\pip\cache\wheels\d4\5a\db\e442580c22be34f69e537448832d7e1ee5a9c5adb63ace30bf
  Created wheel for roundrobin: filename=roundrobin-0.0.4-py3-none-any.whl size=4034 sha256=29a79cfc5213a8c3af56b4c077d486a2cf84298f2693720d517f511b5d540b3f
  Stored in directory: c:\users\mashk\appdata\local\pip\cache\wheels\f0\a5\95\f5228e0966ecd65b54419065aefc1892af1fc06652fcb9f4d0
Successfully built Flask-BasicAuth roundrobin


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [31]:
dtrain = xgb.DMatrix(x_train, label=y_train)

xgb_params = {
    'eta': 0.1, 
    'max_depth': 3,
    'min_child_weight': 1,

    'objective': 'binary:logistic',
    'eval_metric': 'auc',

    'nthread': 8,
    'seed': 1,
    'verbosity': 1,
}

model = xgb.train(xgb_params, dtrain, num_boost_round=175)

In [32]:
import bentoml

In [33]:
bentoml.xgboost.save_model(
    'credit_risk_model',
    model,
    custom_objects={
        'dictVectorizer': dv
    })

Model(tag="credit_risk_model:geedfkks5kj27lcq", path="C:\Users\mashk\bentoml\models\credit_risk_model\geedfkks5kj27lcq\")

In [34]:
import json
request = df_test.iloc[0].to_dict()
print(json.dumps(request, indent=2))

{
  "seniority": 3,
  "home": "owner",
  "time": 36,
  "age": 26,
  "marital": "single",
  "records": "no",
  "job": "freelance",
  "expenses": 35,
  "income": 0.0,
  "assets": 60000.0,
  "debt": 3000.0,
  "amount": 800,
  "price": 1000
}
